Assignment 4 : Colin Kirby

In [1]:
# Imports.
import pygame
import moderngl
import glm
import numpy as np
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame and OpenGL like in prev. Assignments.
def init_window(width=800, height=600):
    pygame.init()
    pygame.display.set_mode((width, height), pygame.OPENGL | pygame.DOUBLEBUF)
    pygame.display.set_caption("Assignment 4: Colin Kirby")
    
    # Create and Set up Context.
    ctx = moderngl.create_context()
    ctx.enable(moderngl.CULL_FACE)
    ctx.enable(moderngl.DEPTH_TEST)
    
    print("Context initialized with depth test and cull face enabled.")
    
    return ctx

In [3]:
# Initialize Window.
ctx = init_window()

Context initialized with depth test and cull face enabled.


In [4]:
# Dictionary to set key inp to Obj.
my_objects = {
    pygame.K_1: "teapot.obj",
    pygame.K_2: "4_tetrahedron.obj",
    pygame.K_3: "6_hexahedron.obj",
    pygame.K_4: "8_octahedron.obj",
    pygame.K_5: "12_dodecahedron.obj",
    pygame.K_6: "20_icosahedron.obj"
}

In [5]:
# Set Default to Teapot.
current_obj = "teapot.obj"
vertex_data, scene_bound = getObjectData(current_obj, normal=True)

Normal exists


In [6]:
#Debugging: Print vertex data length
#print("Vertex Data Length:", len(vertex_data))
#if len(vertex_data) % 6 != 0:
    #print("Error: Vertex data length is not a multiple of 6, normals might be missing!")

In [7]:
# Global VAO obj and Program.
vao = None
program = None

In [8]:
# Upload the vertex data to the GPU
def set_vert_data():
    global vao, program, vertex_data
    print("Uploading vertex data to GPU...")
    
    try:
        # Ensure vertex_data is a numpy array of float32
        vertex_data = np.array(vertex_data, dtype='f4')
        
        # Create a VBO with the current object's data
        vbo = ctx.buffer(vertex_data.tobytes())
        
        # Compile shader program
        program = ctx.program(
            vertex_shader="""
            #version 330
            in vec3 in_position;
            in vec3 in_normal;
            uniform mat4 model;
            uniform mat4 view;
            uniform mat4 proj;
            out vec3 frag_normal;

            void main() {
                frag_normal = in_normal;
                gl_Position = proj * view * model * vec4(in_position, 1.0);
            }
            """,
            fragment_shader="""
            #version 330
            in vec3 frag_normal;
            out vec4 frag_color;

            void main() {
                vec3 color = normalize(frag_normal) * 0.5 + 0.5;  // Compute color from normal
                frag_color = vec4(color, 1.0);
            }
            """
        )
    
        
        # Define the vertex layout (interleaving position and normal data)
        vao = ctx.vertex_array(
            program,
            [
                (vbo, '3f 3f', 'in_position', 'in_normal')  # 3 floats for position, 3 for normal
            ]
        )
        
    
    except Exception as e:
        print("Error uploading vertex data:", e)

In [9]:
# Upload the vertex data for the initial object
set_vert_data()

Uploading vertex data to GPU...


In [10]:
# Create projection and view matrices
fov = 45.0
width, height = 800, 600
aspect_ratio = width / height
near_plane = 0.1
far_plane = 100.0

proj_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near_plane, far_plane)

In [11]:
view_matrix = glm.lookAt(
    glm.vec3(-5.0, 3.0, 10.0),
    glm.vec3(0.0, 0.0, 0.0),
    glm.vec3(0.0, 1.0, 0.0) 
)

In [12]:
# Func for handling diff Inp and Loading Objs.
def handle_keypress(event):
    global current_obj, vertex_data, scene_bound
    if event.key in my_objects:
        current_obj = my_objects[event.key]
        vertex_data, scene_bound = getObjectData(current_obj, normal=True)
        set_vert_data()

In [13]:
# Just translations bc Teapot was positioned higher than other objects for some reason.
translations = {
    "4_tetrahedron.obj": glm.vec3(0.0, 0.0, 0.0),  
    "6_hexahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "8_octahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "12_dodecahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "20_icosahedron.obj": glm.vec3(0.0, 0.0, 0.0),
    "teapot.obj": glm.vec3(0.0, -1.0, 0.0),
}

In [14]:
# Global Vars.
global vao, program, current_obj
    
running = True
    
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            handle_keypress(event)

    # Set up Matrix.
    model_matrix = glm.mat4(1.0)
        
    # Using the Translations Dictionary.
    translation_vector = translations.get(current_obj, glm.vec3(0.0, 0.0, 0.0))
    model_matrix = glm.translate(model_matrix, translation_vector)

    # Write Matrices to Shader.
    if vao is not None and program is not None:
        try:
            # Convert Matrices to NP Arrays.
            model_bytes = np.array(model_matrix, dtype='f4').T.flatten().tobytes()
            view_bytes = np.array(view_matrix, dtype='f4').T.flatten().tobytes()
            proj_bytes = np.array(proj_matrix, dtype='f4').T.flatten().tobytes()
                
            # Write to Uniforms.
            program['model'].write(model_bytes)
            program['view'].write(view_bytes)
            program['proj'].write(proj_bytes)
                
        except Exception as e:
            pass 

    # Clear Screen and Render.
    try:
        ctx.clear(0.1, 0.1, 0.1)
        if vao is not None:
            vao.render(moderngl.TRIANGLES)
        
    except Exception as e:
        pass  # Handle rendering exceptions silently
        
    pygame.display.flip()

pygame.quit()

Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal exists
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Uploading vertex data to GPU...
Normal computed.
Upload